# Policy Evaluation

## L'algorithme

L'algorithme de policy evaluation consiste à calculer la value function d'une policy donnée pour chaque state $s$. Plus formellement, il s'agit d'une application iterative de l'équation d'expectation de Bellman, montrée ci-dessous. Cette équation donne une définition de $v_{π}(s)$, l'expected return à compter du state $s$.

\begin{equation*}
\large v_\pi(s) = \sum_{a} \pi(a|s) \sum_{s'} p(s'|s,a) [R_{ss'}^a + γ v_\pi(s')]
\end{equation*}

<center><em><span style="color:gray">Bellman's expectation equation</span></em></center>

La mise à jour de la value function $V$ se fait de façon asynchrone (voir `Notes`) de la manière suivante :

\begin{align}
\large V(s) \; & \large \leftarrow \sum_{a} \pi(a|s) \sum_{s'} p(s'|s,a) [R_{ss'}^a + γ V(s')]
\end{align}

Voici le pseudocode de l'algorithme :

<img src="img/pseudocode_policyevaluation.png" width="600" height="250" align="center"/>

<center><em><span style="color:gray">Sutton and Barto, chapitre 4</span></em></center>

## Exemple Gridworld

L'environnement utilisé ici est tiré du chapitre 4 du livre (exemple 4.1). Il s'agit d'un gridworld dans lequel l'agent peut exécuter 4 actions déterministiques : haut, droite, bas, gauche. Quelque soit le state de l'agent, celui recevra un reward de -1 après avoir exécuter n'importe quelle action. Toutefois, si l'agent atteint les cases 0 ou 15, l'épisode est terminé. Ainsi, si un agent veut maximiser les rewards obtenus au cours d'un épisode, il doit aller au plus vite sur la case 0 ou 15. Le state de départ de l'agent est random, bien qu'il ne peut pas être 0 ni 15.

<img src="img/gridworld_states.png" width="250" height="250" align="center"/>

In [1]:
import numpy as np

import envs.gridworld_dennybritz as grd

In [2]:
env = grd.GridworldEnv()

### Helper function

#### Cette fonction sert d'interface avec l'environnement.
* `compute_q_value_for_s_a(env, V, s, a, gamma)` retourne, pour le state $s$ et l'action $a$ spécifiés la valeur $Q(s,a)$ soit $\sum_{s'} p(s'|s,a) [R_{ss'}^a + \gamma V(s')]$. <br>Cette fonction interroge la fonction `P[s][a]` de l'environnement qui renvoit une liste de tuple de la forme : `(p(s'|s,a), s', r(s,a,s'), done?)`. L'algorithme loop sur ces tuples et ajoute à `q` (équivalent à $Q(s,a)$) la valeur $p(s'|s,a) [R_{ss'}^a + \gamma V(s')]$.

In [7]:
def compute_q_value_for_s_a(env, V, s, a, gamma):
    q = 0
    
    for p_sPrime, sPrime, r_ss_a, done in env.P[s][a]:
        q += p_sPrime * (r_ss_a + gamma * V[sPrime])
        
    return q

### Initialization

La policy $\pi$ à évaluer est initalizée, donnant à chacune des 4 actions une probabilité de $0.25$ à être exécutée, dans tout state $s$. $\pi$ est ici une table de dimensions $|S|x|A|$. Ainsi, la probabilité d'exécuter $a$ dans le state $s$ est stored dans `pi[s][a]`.

La value function $V$ est initalizée randomly, ici elle attribue une valeur de 0 à tous les states.

Les hyperparamètres $\gamma$ and $\theta$ sont également initializés. Le discount factor prend la valeur de 1, cela ne posera pas de problème car l'horizon est dans ce cas là fini. Le threshold $\theta$ détermine l'écart maximal entre deux mises à jour de value function à partir duquel l'algorithme aura considéré qu'il a convergé sur $v_\pi$.

In [4]:
pi = np.ones([env.nS, env.nA]) * 0.25

V = np.zeros([env.nS, 1])

gamma = 1.0
theta = 0.00001

### Iterations

Pour chaque iteration, pour chaque state $s$, l'algorithme doit mettre à jour $V(s)$ :

\begin{align}
\large V(s) \; & \large \leftarrow \sum_{a} \pi(a|s) Q(s, a)
\\
& \large \leftarrow \sum_{a} \pi(a|s) \sum_{s'} p(s'|s,a) [R_{ss'}^a + \gamma V(s')]
\end{align}

Ainsi, pour chaque state $s$, on doit calculer pour chaque action $a$ la valeur $Q(s, a)$ renvoyée par la fonction `compute_q_value_for_s_a`. 

$Q(s,a)$ sera ensuite ajouté à la nouvelle valeur $V(s)$, mais sera d'abord multiplié par la probabilité d'arriver $\pi(a|s)$, `pi[s][a]`.

L'algorithme est automatiquement arrêté lorsque l'écart maximum entre $V_{k+1}$ et $V_k$ atteint $\theta$.

In [5]:
k=0
while True:
    k+=1
    delta = 0
    
    for s in range(env.nS):
        V_new = 0
        
        for a in range(env.nA):
            prob_a = pi[s][a]
            q_s_a = compute_q_value_for_s_a(env, V, s, a, gamma)
            
            V_new += prob_a * q_s_a
        
        delta = max(delta, np.abs(V_new - V[s]))
        V[s] = V_new

    if(delta < theta) :
        print("Finished after " + str(k) + " iterations")
        break

Finished after 141 iterations


La value function obtenue est affichée ci-dessous.
Les states les plus proches des cases 0 et 15 ont bien la plus grande value. Comme le reward est -1 pour chaque action, on peut interpréter ces résultats comme étant le nombre de steps moyen avant d'atteindre le state terminal, en suivant la policy random $\pi$ donnée.

<img src="img/gridworld_states_values.png" width="250" height="250" align="center"/>

In [6]:
V

array([[  0.        ],
       [-13.99993529],
       [-19.99990698],
       [-21.99989761],
       [-13.99993529],
       [-17.9999206 ],
       [-19.99991379],
       [-19.99991477],
       [-19.99990698],
       [-19.99991379],
       [-17.99992725],
       [-13.99994569],
       [-21.99989761],
       [-19.99991477],
       [-13.99994569],
       [  0.        ]])